# Import Libraries

In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.naive_bayes import *
from sklearn.dummy import *
from sklearn.ensemble import *
from sklearn.neighbors import *
from sklearn.tree import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.calibration import *
from sklearn.linear_model import *
from sklearn.multiclass import *
from sklearn.svm import *

In [3]:
df = pd.read_csv('data/sms-spam-collection-dataset/spam.csv', encoding = "latin-1")
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


### Removing column 2, 3 and 4 as they have no useful information

In [4]:
df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis=1, inplace=True)
df['SMS'] = df['v2']
df['label'] = df['v1'].map({'ham': 0, 'spam': 1})
df.drop(['v1', 'v2'], axis=1, inplace=True)
train_data = df[:4400]
test_data = df[4400:]
df.head()

,SMS,label
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0


In [5]:
def perform(classifiers, vectorizers, train_data, test_data):
    max_score = 0
    max_name = 0
    for classifier in classifiers:
        for vectorizer in vectorizers:
        
            # train
            vectorize_text = vectorizer.fit_transform(train_data.SMS)
            classifier.fit(vectorize_text, train_data.label)

            # score
            vectorize_text = vectorizer.transform(test_data.SMS)
            score = classifier.score(vectorize_text, test_data.label)
            name = classifier.__class__.__name__ + ' with ' + vectorizer.__class__.__name__ 
            print(name, score)
        if score > max_score:
            max_score = score
            max_name = name
    print ('===========================================')
    print ('===========================================')
    print (max_name, max_score)
    print ('===========================================')
    print ('===========================================')

### List of various classifiers

In [6]:
classifiers = [
        BernoulliNB(),
        RandomForestClassifier(n_estimators=100, n_jobs=-1),
        AdaBoostClassifier(),
        BaggingClassifier(),
        ExtraTreesClassifier(),
        GradientBoostingClassifier(),
        DecisionTreeClassifier(),
        CalibratedClassifierCV(),
        DummyClassifier(),
        PassiveAggressiveClassifier(),
        RidgeClassifier(),
        RidgeClassifierCV(),
        SGDClassifier(),
        OneVsRestClassifier(SVC(kernel='linear')),
        OneVsRestClassifier(LogisticRegression()),
        KNeighborsClassifier()
    ]

### List of various vectorizers

In [7]:
vectorizers = [
        CountVectorizer(),
        TfidfVectorizer(),
        HashingVectorizer()
    ]

### Performing classification and save results to a new dataframe

In [8]:
perform(
    classifiers,
    vectorizers,
    train_data,
    test_data
)

BernoulliNB with CountVectorizer 0.9778156996587031
BernoulliNB with TfidfVectorizer 0.9778156996587031
BernoulliNB with HashingVectorizer 0.8728668941979523
RandomForestClassifier with CountVectorizer 0.9795221843003413
RandomForestClassifier with TfidfVectorizer 0.9744027303754266
RandomForestClassifier with HashingVectorizer 0.9684300341296929
AdaBoostClassifier with CountVectorizer 0.9718430034129693
AdaBoostClassifier with TfidfVectorizer 0.9692832764505119
AdaBoostClassifier with HashingVectorizer 0.9735494880546075
BaggingClassifier with CountVectorizer 0.9701365187713311
BaggingClassifier with TfidfVectorizer 0.9692832764505119
BaggingClassifier with HashingVectorizer 0.9692832764505119
ExtraTreesClassifier with CountVectorizer 0.9812286689419796
ExtraTreesClassifier with TfidfVectorizer 0.9786689419795221
ExtraTreesClassifier with HashingVectorizer 0.9667235494880546
GradientBoostingClassifier with CountVectorizer 0.9692832764505119
GradientBoostingClassifier with TfidfVectori

In [9]:
# train the classifier with best accuracy
Classifier = OneVsRestClassifier(SVC(kernel='linear', probability = True))
Vectorizer = TfidfVectorizer()
vectorize_text = Vectorizer.fit_transform(train_data.SMS)
Classifier.fit(vectorize_text, train_data.label)

OneVsRestClassifier(estimator=SVC(kernel='linear', probability=True))

In [10]:
SMS = ' won a 1 week FREE membership in our $100,000 Prize Jackpot! Txt the word: C'
vectorize_message = Vectorizer.transform([SMS])
predict = Classifier.predict(vectorize_message)[0]

In [11]:
if predict == 0:
    print ('ham')
else:
    print ('spam')

spam
